In [1]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI


In [2]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()



In [4]:
OPENAI_API_KEY = ""

In [5]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [7]:
chain = load_summarize_chain(llm,chain_type="stuff")

In [8]:
chain.run(docs)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The article discusses the concept of building autonomous agents powered by Large Language Models (LLMs). It explores the components of such agents, including planning, memory, and tool use, with examples such as scientific discovery agents and generative agents simulation. Challenges such as finite context length, reliability of natural language interface, and long-term planning are highlighted. The article also references various studies and projects in this domain.'

In [9]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

In [10]:
prompt_template = """
Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
prompt = PromptTemplate.from_template(prompt_template)

In [11]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [12]:
llm_chain = LLMChain(llm=llm,prompt=prompt)

In [15]:
stuff_chain  = StuffDocumentsChain(llm_chain=llm_chain,document_variable_name='text')
stuff_chain.run(docs)

'The article discusses the concept of building autonomous agents powered by large language models (LLMs), focusing on planning, memory, and tool use components. It includes examples like AutoGPT, GPT-Engineer, and generative agents simulation. Challenges include finite context length, reliability of natural language interface, and long-term planning difficulties. The article provides a detailed overview of the key components and case studies, highlighting the potential of LLMs in creating powerful problem-solving agents.'

In [16]:
#map_reduce approach

In [18]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""

prompt = PromptTemplate.from_template(map_template)
prompt

PromptTemplate(input_variables=['docs'], template='The following is a set of documents\n{docs}\nBased on this list of docs, please identify the main themes \nHelpful Answer:')

In [19]:
map_chain = LLMChain(llm=llm,prompt=prompt)

In [21]:
!pip install langchainhub

  Using cached langchainhub-0.1.15-py3-none-any.whl.metadata (621 bytes)
Using cached langchainhub-0.1.15-py3-none-any.whl (4.6 kB)


In [22]:
from langchain import hub

map_prompt = hub.pull("rlm/map-prompt")

In [23]:
map_prompt

ChatPromptTemplate(input_variables=['docs'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'map-prompt', 'lc_hub_commit_hash': 'de4fba345f211a462584fc25b7077e69c1ba6cdcf4e21b7ec9abe457ddb16c87'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], template='The following is a set of documents:\n{docs}\nBased on this list of docs, please identify the main themes \nHelpful Answer:'))])

In [24]:
# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [26]:
reduce_chain = LLMChain(llm=llm,prompt=reduce_prompt)

In [27]:
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [28]:
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="docs",
)

In [29]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 1003, which is longer than the specified 1000


In [30]:
map_reduce_chain.run(split_docs)

'The main themes identified across the provided documents include building autonomous agents powered by large language models (LLMs), components of LLM-powered agents such as planning, memory, and tool use, task decomposition, self-reflection, and memory types in agent systems, case studies and proof-of-concept examples of LLM-powered agents, challenges in building and utilizing LLM-powered agents, memory optimization techniques like MIPS and LSH, algorithms for nearest neighbor search, neuro-symbolic architecture for agents, fine-tuning language models with external tools, task planning and model selection using language models, and challenges and limitations in the development and application of LLM-powered agents. These themes collectively highlight the advancements, applications, and challenges in leveraging LLMs for building intelligent autonomous systems.'

In [31]:
#refine method 

In [32]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

In [33]:
refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Italian"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)

In [34]:
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [35]:
result

{'intermediate_steps': ['The article discusses the concept of building autonomous agents powered by LLM (large language models) and highlights key components such as planning, memory, and tool use. It explores proof-of-concept examples like AutoGPT and GPT-Engineer, demonstrating the potential of LLM beyond generating text and programs. The agent system leverages LLM for problem-solving and task decomposition, self-reflection, and external tool use, making it a powerful general problem solver.',
  "L'articolo discute il concetto di costruire agenti autonomi alimentati da grandi modelli linguistici (LLM) e mette in evidenza componenti chiave come la pianificazione, la memoria e l'uso degli strumenti. Esplora esempi di prova concettuali come AutoGPT e GPT-Engineer, dimostrando il potenziale degli LLM oltre alla generazione di testi e programmi. Il sistema di agenti sfrutta gli LLM per la risoluzione di problemi e la decomposizione delle attività, l'auto-riflessione e l'uso di strumenti e